In [1]:
import pandas as pd
import numpy as np

train=pd.read_csv('train_features.csv')
train_labels=pd.read_csv('train_labels.csv')
test=pd.read_csv('test_features.csv')
submission=pd.read_csv('sample_submission.csv')

In [2]:
if 'label' in train.columns:
    train.drop(columns=['label'], inplace=True)
train_total = pd.merge(train, train_labels, on='id')

In [3]:
labels = train_labels['label'].drop_duplicates()

In [4]:
len(train_total)

1875000

In [5]:
len(train_labels)*600

1875000

In [6]:
aug_cnt = 1400

for idx, label in labels.iteritems():
    tmps = train_total[train_total['label'] == label].copy()
    while len(train_labels[train_labels['label'] == label]) < aug_cnt:
        id_list = tmps['id'].drop_duplicates()
        print(label,":", len(train_labels[train_labels['label'] == label]))
        for idx, _id in id_list.iteritems():
            if len(train_labels[train_labels['label'] == label]) < aug_cnt:
                tmp = tmps[tmps['id'] == _id].copy()
                tmp['acc_x'] = tmp['acc_x'] + tmp['acc_x'].std() * np.random.normal(0.0, 1.0, 600)
                tmp['acc_y'] = tmp['acc_y'] + tmp['acc_y'].std() * np.random.normal(0.0, 1.0, 600)
                tmp['acc_z'] = tmp['acc_z'] + tmp['acc_z'].std() * np.random.normal(0.0, 1.0, 600)
                tmp['gy_x'] = tmp['gy_x'] + tmp['gy_x'].std() * np.random.normal(0.0, 1.0, 600)
                tmp['gy_y'] = tmp['gy_y'] + tmp['gy_y'].std() * np.random.normal(0.0, 1.0, 600)
                tmp['gy_z'] = tmp['gy_z'] + tmp['gy_z'].std() * np.random.normal(0.0, 1.0, 600)
                newid = train['id'].max() + 1
                tmp['id'] = newid
                train = pd.concat([train, tmp])
                train_labels = train_labels.append({'id': newid, 'label': label}, ignore_index=True)

            if len(train_labels[train_labels['label'] == label]) < aug_cnt:
                for rate in [0.1*i for i in range(5, 15)]:
                    tmp = tmps[tmps['id'] == _id].copy()
                    tmp['acc_x'] = tmp['acc_x']*rate
                    tmp['acc_y'] = tmp['acc_y']*rate
                    tmp['acc_z'] = tmp['acc_z']*rate
                    tmp['gy_x'] = tmp['gy_x']*rate
                    tmp['gy_y'] = tmp['gy_y']*rate
                    tmp['gy_z'] = tmp['gy_z']*rate
                    newid = train['id'].max() + 1
                    tmp['id'] = newid
                    train = pd.concat([train, tmp])
                    train_labels = train_labels.append({'id': newid, 'label': label}, ignore_index=True)

            if len(train_labels[train_labels['label'] == label]) < aug_cnt:
                tmp = tmps[tmps['id'] == _id].copy()
                tmp.sort_values(by='time', ascending=False)
                newid = train['id'].max() + 1
                tmp['id'] = newid
                train = pd.concat([train, tmp])
                train_labels = train_labels.append({'id': newid, 'label': label}, ignore_index=True)
   
    print(label)
    train.to_csv(f'./aug/train_features_{label}.csv')
    train_labels.to_csv(f'./aug/train_labels_{label}.csv')

37 : 35
37 : 455
37 : 875
37 : 1295
37


FileNotFoundError: [Errno 2] No such file or directory: '/aug/train_features_37.csv'

In [ ]:
train_feature = pd.DataFrame()
train_label = pd.DataFrame()

for _, label in labels.iteritems():
    tmp_feature = pd.read_csv(f"aug/train_features_{label}.csv")
    pd.concat([train_feature, tmp])

    tmp_label = pd.read_csv(f"aug/train_labels_{label}.csv")
    pd.concat([train_label, tmp])



In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D

In [ ]:
X=tf.reshape(np.array(train_feature.iloc[:,2:]),[-1, 600, 6])
print(X.shape)

y=tf.keras.utils.to_categorical(train_label['label']) 
print(y.shape)

In [ ]:
class InceptionModule(tf.keras.layers.Layer):
    def __init__(self, num_filters=32, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.num_filters = num_filters
        self.activation = tf.keras.activations.get(activation)

    def _default_Conv1D(self, filters, kernal_size):
        return tf.keras.layers.Conv1D(filters=filters,
                                      kernal_size=kernal_size,
                                      strides=1,
                                      activation='relu',
                                      use_bias=False)

    def call(self, inputs):
        
        z_bottleneck = self._default_Conv1D(filters=self.num_filters, kernal_size=1)(inputs)
        z_maxpool = tf.keras.layers.MaxPool1D(pool_size=3, strides=1, padding='same')(inputs)

        z1 = self._default_Conv1D(filters=self.num_filters, kernel_size=10)(z_bottleneck)
        z2 = self._default_Conv1D(filters=self.num_filters, kernel_size=20)(z_bottleneck)
        z3 = self._default_Conv1D(filters=self.num_filters, kernel_size=40)(z_bottleneck)
        z4 = self._default_Conv1D(filters=self.num_filters, kernel_size=1)(z_maxpool)

        z = tf.keras.layers.Concatenate(axis=2)([z1, z2, z3, z4])
        z = tf.keras.layers.BatchNormalization()(z)

        return self.activation(z)


In [ ]:
def shortcut_layer(inputs, z_inception):
    z_shortcut = tf.keras.layers.Conv1D(filters=int(out_tensor.shape[-1]), 
                                        kernel_size=1, 
                                        padding='same', 
                                        use_bias=False)(inputs)

    z_shortcut = tf.keras.layers.BatchNormalization()(z_shortcut)

    z = keras.layers.Add()([z_shortcut, z_inception])

    return tf.keras.layers.Activation('relu')(z)


def build_model(input_shape, num_classes, num_modules=6):
    input_layer = tf.keras.layers.Input(input_shape)
    z = input_layer
    z_residual = input_layer

    for i in range(num_modules):
        z = InceptionModule()(z)
        if i%3 == 2:
            z = shortcut_layer(z_residual, z)
            z_residual = z

    gap_layer = tf.keras.layers.GlobalAveragePooling1D()(z)
    output_layer = tf.keras.layers.Dense(num_classes, activation='softmax')(gap_layer)

    model = tf.keras.models.Model(inputs= input_layer, outputs= output_layer)
    model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])

    return model